In [313]:
import boto3

In [314]:
bd_clt = boto3.client("bedrock-runtime", region_name="us-east-1")

In [315]:
question = "User: Find all employees with the last name \"ERIC\" who are enrolled in a service?"

In [316]:
prompt = """
You are an AI SQL assistant. Generate SQL queries based on natural language prompts. Use the given schema as reference. 
Also append within quotes the database name with a dot in outputquery.

Schema:
{
  "tables": {
    "employee": {
      "columns": {
        "employee_id": "int",
        "first_name": "varchar",
        "last_name": "varchar",
        "department_id": "int",
        "hire_date": "date"
      }
    },
    "department": {
      "columns": {
        "department_id": "int",
        "department_name": "varchar"
      }
    },
    "services": {
      "columns": {
        "service_id": "int",
        "service_name": "varchar"
      }
    },
    "enrollment": {
      "columns": {
        "employee_id": "int",
        "service_id": "int",
        "enrollment_date": "date"
      }
    }
  }
}

Examples:
User: List all employees in the IT department.
AI: SELECT * FROM "employee" e JOIN department d ON e.department_id = d.department_id WHERE d.department_name = 'IT';

User: Get all employees hired after January 1, 2021.
AI: SELECT * FROM "employee" WHERE hire_date > "2021-01-01";

User: Get all services available in the system.
AI: SELECT * FROM "services";

User: Find all employees with the last name 'Smith'.
AI: SELECT * FROM "employee" WHERE last_name = 'Smith';

User: Get the department details of employees hired before 2020.
AI: SELECT e.first_name, e.last_name, d.department_name FROM "employee" e JOIN department d ON e.department_id = d.department_id WHERE e.hire_date < '2020-01-01';

User: List all services employees have enrolled in.
AI: SELECT s.service_name, e.first_name, e.last_name FROM "employee" en JOIN services s ON en.service_id = s.service_id JOIN employee e ON en.employee_id = e.employee_id;

User: Which employees are enrolled in the Health Insurance service?
AI: SELECT e.first_name, e.last_name FROM "employee" en JOIN services s ON en.service_id = s.service_id JOIN employee e ON en.employee_id = e.employee_id WHERE s.service_name = 'Health Insurance';

User: What is the total number of employees in each department?
AI: SELECT d.department_name, COUNT(e.employee_id) AS employee_count FROM "employee" e JOIN department d ON e.department_id = d.department_id GROUP BY d.department_name;

User: Find employees who have never enrolled in a service.
AI: SELECT e.first_name, e.last_name FROM "employee" e LEFT JOIN enrollment en ON e.employee_id = en.employee_id WHERE en.employee_id IS NULL;

User: Which employees are enrolled in multiple services?
AI: SELECT e.first_name, e.last_name, COUNT(en.service_id) AS service_count FROM "enrollment" en JOIN employee e ON en.employee_id = e.employee_id GROUP BY e.first_name, e.last_name HAVING COUNT(en.service_id) > 1;

User: What is the total number of enrollments per service?
AI: SELECT s.service_name, COUNT(en.service_id) AS total_enrollments FROM "enrollment" en JOIN services s ON en.service_id = s.service_id GROUP BY s.service_name;

Now, generate the SQL query for the following request:
User: Find all employees?
AI:
"""

In [317]:
model_id = "amazon.titan-text-express-v1" # "amazon.titan-embed-text-v2:0" # "anthropic.claude-3-5-haiku-20241022-v1:0" #"anthropic.claude-3-haiku-20240307-v1:0"  #"anthropic.claude-3-5-haiku-20241022-v1:0"
#prompt = "Give me the current date time in EST"
native_request = {
 "inputText": prompt,
    "textGenerationConfig": {
        "maxTokenCount": 512,
        "temperature": 0.5,
    }
}
import json
request = json.dumps(native_request)
response = bd_clt.invoke_model(modelId =model_id, body = request)
output = json.loads(response['body'].read())
print(output)
sql_stmt = output['results'][0]['outputText']
print(sql_stmt)

{'inputTextTokenCount': 965, 'results': [{'tokenCount': 9, 'outputText': 'SELECT * FROM "employee";', 'completionReason': 'FINISH'}]}
SELECT * FROM "employee";


In [318]:
athena_client = boto3.client("athena", region_name = 'us-east-1')

bucket_name = "query-gen-hackathon"


In [319]:
response = athena_client.start_query_execution(QueryString= sql_stmt, QueryExecutionContext = {'Database': 'finetune_llm_querygen', 'Catalog': 'AwsDataCatalog'},
                                              ResultConfiguration = {'OutputLocation': "s3://query-gen-hackathon/athena-query-result", 
                                                                    })
query_id = response['QueryExecutionId']
print(query_id)

5115281b-a198-4440-8c50-2310d62a2817


In [325]:
exec_status = athena_client.get_query_execution(QueryExecutionId=query_id)
print(exec_status)
status = exec_status['QueryExecution']['Status']['State']
print(status)

{'QueryExecution': {'QueryExecutionId': '5115281b-a198-4440-8c50-2310d62a2817', 'Query': 'SELECT * FROM "employee"', 'StatementType': 'DML', 'ResultConfiguration': {'OutputLocation': 's3://query-gen-hackathon/athena-query-result/5115281b-a198-4440-8c50-2310d62a2817.csv'}, 'ResultReuseConfiguration': {'ResultReuseByAgeConfiguration': {'Enabled': False}}, 'QueryExecutionContext': {'Database': 'finetune_llm_querygen', 'Catalog': 'awsdatacatalog'}, 'Status': {'State': 'SUCCEEDED', 'SubmissionDateTime': datetime.datetime(2025, 3, 3, 16, 54, 27, 825000, tzinfo=tzlocal()), 'CompletionDateTime': datetime.datetime(2025, 3, 3, 16, 54, 28, 923000, tzinfo=tzlocal())}, 'Statistics': {'EngineExecutionTimeInMillis': 714, 'DataScannedInBytes': 3194, 'TotalExecutionTimeInMillis': 1098, 'QueryQueueTimeInMillis': 114, 'ServicePreProcessingTimeInMillis': 203, 'QueryPlanningTimeInMillis': 77, 'ServiceProcessingTimeInMillis': 67, 'ResultReuseInformation': {'ReusedPreviousResult': False}}, 'WorkGroup': 'prim

In [321]:
# results = athena_client.get_query_results(
#     QueryExecutionId=query_id,   
#     MaxResults=123
# )
# import pandas as pd
# print(results)
# data = results['ResultSet']
# print(data)
# df = pd.DataFrame.from_dict(data['Rows'])
# print(df)

In [326]:
import time
print('starting')
print(query_id)
while True:
    print('checking')
    if status == 'QUEUED' or status == 'RUNNING':
        print('waiting')
        time.sleep(1)        
    elif status == 'SUCCEEDED':
        print('done')
        s3_client = boto3.client('s3')
        resp = s3_client.get_object(Bucket = bucket_name, Key = f'athena-query-result/{query_id}.csv')
        print(resp['Body'])
        break


starting
5115281b-a198-4440-8c50-2310d62a2817
checking
done


In [327]:
#import pandas as pd
output = pd.read_csv(resp['Body'])
print("****")
print(output)

****
    employee_id first_name last_name  department_id   hire_date
0             1      Bryan      Diaz            103  2020-08-10
1             2     Marvin     Drake            101  2023-12-18
2             3     Trevor  Williams            102  2021-01-18
3             4     Angela  Anderson            102  2021-02-19
4             5   Michelle    Guzman            102  2022-02-22
..          ...        ...       ...            ...         ...
95           96      Kevin      Ford            103  2020-02-12
96           97      Julie    Moreno            101  2021-01-18
97           98   Michelle   Fleming            101  2024-09-23
98           99    Jeffery   Bennett            101  2021-06-10
99          100    Jasmine   Jackson            102  2021-08-05

[100 rows x 5 columns]
